In [46]:
## Importing the Libraries, Package 

In [47]:
import requests
from bs4 import BeautifulSoup
import re
import os 
import time
from io import BytesIO
from zipfile import ZipFile
import pandas as pd
import glob
# import matplotlib.pyplot as plt
# import seaborn as sns
import numpy as np
import sys
print('Step1. Import Done ')

Step1. Import Done 


In [48]:
fileDir = os.path.dirname(os.path.realpath('__file__'))
print('Step2. Local Path defined')
print(fileDir)

Step2. Local Path defined
C:\Users\mohan\Downloads\Ads_MidtermU\Ads_Midterm


In [49]:
baseUrl='https://freddiemac.embs.com/FLoan/'
postUrl='Data/download.php'

def createCredentialData(user, passwd):
    creds={'username': user,'password': passwd}
    return creds

def getFiles(cred):
    ## We are using inside WITH BLock so that session is closed ASAP with BLock is exited 
    with requests.Session() as s:
        ## Step 1 routing to auth.php Site with the proper crentials 
        urlOne = s.post(baseUrl+"secure/auth.php", data=cred) 
        print(urlOne.text)
        if "Please log in" in urlOne.text:
        ## IF CREDENTIALS are not valid Throw Alert 
            print("Alert: Invalid Credentials, Please try again or sign up on below site \n https://freddiemac.embs.com/FLoan/Bin/loginrequest.php")
        else:
            print("Step1: Logged in")
        ## Step 2 Preparing the data for to Accept terms and Conditions 
            pay2={'accept': 'Yes','acceptSubmit':'Continue','action':'acceptTandC'}
            finalUrl=s.post(baseUrl +"Data/download.php",pay2)
            if "Loan-Level Dataset" in finalUrl.text:
                      print("Step2 : Terms and Conditions Accepted")
                      soup = BeautifulSoup(finalUrl.content, "html.parser")   
                      links_list = soup.findAll('a')
                      print("Step3: Filtered the Sample Files with Condition > 2005")
                      print("Status::::::::::")
                      for ele in links_list:
        ## Filtering the ZIp files >= 2005 
                         if 'sample' in ele.get_text():
                            if(ele.get_text()[-8:-4] >= '2005'):
                                    print(ele.get_text()[-8:-4])
                                    tempUrl = baseUrl+"Data/"+ele.get('href')                         
                                    b =time.time()
                                    downloadUrl=s.post(tempUrl) ## return type = Response
                                    e=time.time()
                                    print(tempUrl + " took "+ str(e-b)+" sec")
                                    with ZipFile(BytesIO(downloadUrl.content)) as zfile:
                                          zfile.extractall(os.path.join(fileDir, 'adsDataRepo/'+'Sample_data_'+ele.get_text()[-8:-4]+'/'))
                                          print("File "+ ele.get_text()+" Downloaded")
    
            else:
                print("Alert: Please Check the rerouting action suffix")
        
        ##To scrape the data from the Site finalUrl.       


    
    
 


In [54]:
frames=[]
frames2=[]

In [69]:
def dataFiltering():
    syears = [syear for syear in os.listdir(fileDir+'/adsDataRepo/')]          
    for syear in syears:
        print(syear)
        ## Checking only the files we want , have to discard .ipynb files 
        
        if ('Sample_data' in syear):
#             if('2008' in syear or '2007' in syear or '2009' in syear):
                    print('#####################################################################################################')
                    print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$4")
                    print(syear)
                    originalf = glob.glob(fileDir+'/adsDataRepo/'+syear+'/*_orig_*.txt')
                    print(originalf)
                    performancef= glob.glob(fileDir+'/adsDataRepo/'+syear+'/*_svcg_*.txt')

                    headerNames1 = ['CreditScore','FirstPaymentDate','FirstTimeHomeBuyerFlag','MaturityDate','MSA','MIP','NumberOfUnits',
                                   'OccupancyStatus','OCLTV','DTI','OriginalUPB','OLTV','OriginalInterestRate','Channel','PrepaymentPenaltyFlag',
                                   'ProductType','PropertyState','PropertyType','PostalCode','LoanSequenceNumber','LoanPurpose',
                                   'OriginalLoanTerm','NumberOfBorrowers','SellerName','ServicerName','SuperConformingFlag']
                    headerNames2 = ['LoanSequenceNumber','MonthlyReportingPeriod','CurrentActualUpb','CurrentLoadDelinquencyStatus',
                                    'LoanAge','RemainingMonthsToLegalMaturity','RepurchaseFlag','ModificationFlag','ZeroBalanceCode',
                                    'ZeroBalanceEffectiveDate','CurrentInterestRate','CurrentDeferredUpb','DueDateOfLastPaidInstallment',
                                    'MiRecoveries','NetSalesProceeds','NonMiRecoveries','Expenses','LegalCosts',
                                    'MaintenanceAndPreservationCosts','TaxesAndInsurance','MiscellaneousExpenses','ActualLossCalculation',
                                    'Modification Cost']


                    with open(originalf[0]) as f:
                          ## Reading the data from .txt file 
                          dataf1 = pd.read_table(f, sep='|', low_memory=False, header=None,lineterminator='\n', names= headerNames1,
                                                 dtype={'CreditScore':int,'OCLTV': str,'OLTV': str,'DTI': str,'CreditScore': str, 'PostalCode': str,
                                                        'SuperConformingFlag' : str})
                          ## Calling Functions to preprocess it   
        #                   cleandata1 = originalDatacleaning(dataf1)
                          ## Saving the clean file in the csv format
        #                   dataf1.to_csv(fileDir+'/adsDataRepo/'+syear+"/Original_Clean_"+syear+".csv")
                          frames.append(dataf1)
                          print("Clean DOriginal Data CSV Created")

                    with open(performancef[0]) as f:
                        ## Reading the data from .txt file 
                          dataf2 = pd.read_table(f, sep='|', low_memory=False,header=None,lineterminator='\n',names= headerNames2,
                                                         dtype={'ZeroBalanceCode':str, 'CurrentLoadDelinquencyStatus':str, 
                                                                     'ModificationFlag':str,'NetSalesProceeds':str, 'LegalCosts':str, 
                                                                     'MaintenanceAndPreservationCosts':str, 'TaxesAndInsurance':str, 
                                                                     'Expenses':str, 'MiscellaneousExpenses':str })
                          ## Calling Functions to preprocess it 
        #                   cleanperf1 = performanceDatacleaning(dataf2)
                          ## Saving the clean file in the csv format
        #                   dataf2.to_csv(fileDir+'/adsDataRepo/'+syear+"/Performance_Clean_"+syear+".csv")
                          frames2.append(dataf2)
                          print("Clean Performance Data CSV Created")
                    print('#####################################################################################################')
                    print('#####################################################################################################')
                  
 


In [70]:
print("Step3. All Functions Defined")

Step3. All Functions Defined


In [71]:
def main():
    
#     creds=createCredentialData("tapadyuti@gmail.com","^zmI1cWr")
#     if len(sys.argv) == 4 and sys.argv[3] in ['0','1']:
        username = raw_input("Username ")
        password = raw_input("password ")
        creds=createCredentialData(username,password)
#         getFiles(creds)
        dataFiltering()
    
if __name__ == '__main__':
    print("Step1.Calling Main function")
        ## Making sure that adsDataRepo be present in the local System , where the data gets downloaded 
    if(not os.path.exists(fileDir+'/adsDataRepo/')):
        os.makedirs(fileDir+'/adsDataRepo/')
    main()   

Step1.Calling Main function
Username l
password l
Historical_data_2008
Sample_data_2005
Sample_data_2006
Sample_data_2007
#####################################################################################################
$$$$$$$$$$$$$$$$$$$$$$$$$$$$4
Sample_data_2007
['C:\\Users\\mohan\\Downloads\\Ads_MidtermU\\Ads_Midterm/adsDataRepo/Sample_data_2007\\sample_orig_2007.txt']
Clean DOriginal Data CSV Created
Clean Performance Data CSV Created
#####################################################################################################
#####################################################################################################
Sample_data_2008
#####################################################################################################
$$$$$$$$$$$$$$$$$$$$$$$$$$$$4
Sample_data_2008
['C:\\Users\\mohan\\Downloads\\Ads_MidtermU\\Ads_Midterm/adsDataRepo/Sample_data_2008\\sample_orig_2008.txt']
Clean DOriginal Data CSV Created
Clean Performance Data CSV Created
###

In [77]:
finalpredictionData = pd.concat(frames)
finalpredictionData.to_csv("PredicitionData789.csv")
finalpredictionData.to_csv("PredicitionData.csv")

finalClassificationData = pd.concat(frames2)
finalClassificationData.to_csv("ClassificationData789.csv")
finalClassificationData.to_csv("ClassificationData.csv")



In [76]:
finalClassificationData.describe()

,MonthlyReportingPeriod,CurrentActualUpb,LoanAge,RemainingMonthsToLegalMaturity,ZeroBalanceEffectiveDate,CurrentInterestRate,CurrentDeferredUpb,DueDateOfLastPaidInstallment,MiRecoveries,NonMiRecoveries,ActualLossCalculation,Modification Cost
count,7.377106e+06,7.377106e+06,7.377106e+06,7.377106e+06,126781.000000,7.377106e+06,7.377106e+06,10539.000000,6450.000000,6450.000000,6450.000000,149989.000000
mean,2.010944e+05,1.703641e+05,3.402513e+01,3.009066e+02,201141.175200,5.709942e+00,3.889761e+02,201136.248221,14407.980155,11192.189767,-83307.397674,828.286578
std,2.290092e+02,1.038132e+05,2.620682e+01,7.348418e+01,205.832254,8.918052e-01,5.842595e+03,262.988133,27509.298948,43363.667445,69154.530920,6569.389196
min,2.007020e+05,0.000000e+00,0.000000e+00,-3.100000e+01,200703.000000,0.000000e+00,0.000000e+00,200705.000000,0.000000,-14677.000000,-525560.000000,-763.100000
25%,2.009080e+05,9.376719e+04,1.300000e+01,2.890000e+02,201004.000000,5.000000e+00,0.000000e+00,200909.000000,0.000000,343.250000,-117233.750000,0.000000
50%,2.010120e+05,1.476199e+05,2.800000e+01,3.260000e+02,201110.000000,5.875000e+00,0.000000e+00,201102.000000,0.000000,1192.000000,-71835.500000,0.000000
75%,2.012110e+05,2.277566e+05,5.100000e+01,3.460000e+02,201302.000000,6.375000e+00,0.000000e+00,201307.000000,20451.000000,2779.500000,-33232.750000,0.000000
max,2.016120e+05,8.020000e+05,1.180000e+02,5.190000e+02,201612.000000,9.125000e+00,3.325000e+05,201610.000000,224716.000000,511239.000000,112666.000000,197681.140000
